In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# Załaduj modele
model_static = load_model('StaticBiLSTM/static_gestures_model.keras')
label_encoder_static = joblib.load('StaticBiLSTM/static_gestures_labels.pkl')

model_dynamic = load_model('DynamicBiLSTM/dynamic_gestures_model.keras')
label_encoder_dynamic = joblib.load('DynamicBiLSTM/dynamic_gestures_labels.pkl')

# Parametry
motion_threshold = 1.8  # Próg rozróżnienia statyczny/dynamiczny
motion_history = []
motion_history_len = 5
cooldown_frames = 0
cooldown_threshold = 10
recognized_sign = ''
recognized_type = ''
last_prediction = None
static_buffer = []
dynamic_buffer = []

# MediaPipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
def normalize_landmarks_buffer(buffer):
    data = []
    for frame_idx, row in enumerate(buffer):
        if len(row) != 63:
            print(f"[WARNING] Frame {frame_idx} has invalid length {len(row)}")
            continue
        frame_data = {'frame': frame_idx}
        for i in range(21):
            frame_data[f'landmark_{i}_x'] = row[i * 3]
            frame_data[f'landmark_{i}_y'] = row[i * 3 + 1]
            frame_data[f'landmark_{i}_z'] = row[i * 3 + 2]
        data.append(frame_data)

    if len(data) == 0:
        print("[WARNING] No valid frames in buffer")
        return np.zeros((1, 63))

    df = pd.DataFrame(data)
    df = df.sort_values("frame").copy()

    wrist_ref = {
        axis: df[df["frame"] == 0].iloc[0][f"landmark_1_{axis}"]
        for axis in ['x', 'y', 'z']
    }

    for axis in ['x', 'y', 'z']:
        for i in range(1, 21):
            col = f'landmark_{i}_{axis}'
            df[col] = df[col] - wrist_ref[axis]

    norm_buffer = []
    for _, row in df.iterrows():
        landmarks = []
        for i in range(21):
            landmarks.extend([
                row[f'landmark_{i}_x'],
                row[f'landmark_{i}_y'],
                row[f'landmark_{i}_z']
            ])
        norm_buffer.append(landmarks)

    norm_array = np.array(norm_buffer)
    print(f"[DEBUG] Normalized buffer shape: {norm_array.shape}")
    return norm_array

def extract_right_hand_landmarks(results):
    if results.right_hand_landmarks:
        return [coord for lm in results.right_hand_landmarks.landmark for coord in (lm.x, lm.y, lm.z)]
    return None

def get_hand_bbox(landmarks, image_width, image_height):
    xs = [lm.x * image_width for lm in landmarks.landmark]
    ys = [lm.y * image_height for lm in landmarks.landmark]
    x_min, x_max = max(int(min(xs)), 0), min(int(max(xs)), image_width)
    y_min, y_max = max(int(min(ys)), 0), min(int(max(ys)), image_height)
    return (x_min, y_min, x_max, y_max)

# Inicjalizacja kamery
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_diff = cv2.absdiff(prev_gray, curr_gray)
        motion_level = np.sum(frame_diff) / (frame.shape[0] * frame.shape[1])
        prev_gray = curr_gray.copy()

        # Wygładzanie ruchu
        motion_history.append(motion_level)
        if len(motion_history) > motion_history_len:
            motion_history.pop(0)
        smooth_motion = np.mean(motion_history)

        print(f"[DEBUG] Motion level: {motion_level:.3f}, Smooth motion: {smooth_motion:.3f}")

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)
        image_height, image_width, _ = frame.shape

        landmarks = extract_right_hand_landmarks(results)

        if landmarks:
            mp_drawing.draw_landmarks(
                frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            bbox = get_hand_bbox(results.right_hand_landmarks, image_width, image_height)
            x1, y1, x2, y2 = bbox
            box_color = (0, 255, 0) if recognized_type == "Statyczny" else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)

            if smooth_motion < motion_threshold:
                dynamic_buffer.clear()
                static_buffer.append(landmarks)
                # print(f"[DEBUG] Static buffer size: {len(static_buffer)}")
                if len(static_buffer) >= 5:
                    if cooldown_frames == 0:
                        # print("[DEBUG] Running static prediction...")
                        norm_static = normalize_landmarks_buffer(static_buffer)
                        if norm_static.shape[0] == 5:
                            norm_static = np.expand_dims(norm_static, axis=0)
                            prediction = model_static.predict(norm_static)[0]
                            label = label_encoder_static.inverse_transform([np.argmax(prediction)])[0]
                            conf = np.max(prediction)
                            print(f"[STATIC] Predicted: {label} (conf: {conf:.2f})")
                            if conf > 0.5 and label != last_prediction:
                               recognized_sign = label
                               recognized_type = "Statyczny"
                               last_prediction = label
                               cooldown_frames = cooldown_threshold
                               static_buffer.clear()
                               dynamic_buffer.clear()
                               print(f"[INFO] Recognized static gesture: {recognized_sign} with confidence {conf:.2f}")
                        else:
                            print("[WARNING] Normalized static buffer shape unexpected:", norm_static.shape)
                        static_buffer.clear()
                    else:
                        print(f"[DEBUG] Static prediction skipped due to cooldown: {cooldown_frames}")
            else:
                static_buffer.clear()
                dynamic_buffer.append(landmarks)
                # print(f"[DEBUG] Dynamic buffer size: {len(dynamic_buffer)}")
                if len(dynamic_buffer) >= 30:
                    if cooldown_frames == 0:
                        # print("[DEBUG] Running dynamic prediction...")
                        norm_dynamic = normalize_landmarks_buffer(dynamic_buffer)
                        if norm_dynamic.shape[0] == 30:
                            norm_dynamic = np.expand_dims(norm_dynamic, axis=0)
                            prediction = model_dynamic.predict(norm_dynamic)[0]
                            label = label_encoder_dynamic.inverse_transform([np.argmax(prediction)])[0]
                            conf = np.max(prediction)
                            print(f"[DYNAMIC] Predicted: {label} (conf: {conf:.2f})")
                            if conf > 0.5 and label != last_prediction:
                                recognized_sign = label
                                recognized_type = "Dynamiczny"
                                last_prediction = label
                                cooldown_frames = cooldown_threshold
                                static_buffer.clear()
                                dynamic_buffer.clear()
                                print(f"[INFO] Recognized dynamic gesture: {recognized_sign} with confidence {conf:.2f}")
                        else:
                            print("[WARNING] Normalized dynamic buffer shape unexpected:", norm_dynamic.shape)
                        dynamic_buffer.clear()
                    else:
                        print(f"[DEBUG] Dynamic prediction skipped due to cooldown: {cooldown_frames}")

            # Rysowanie labela na górze ekranu, wyśrodkowany
            if recognized_sign and recognized_type:
                label_text = f'{recognized_sign} ({recognized_type})'
                label_pos_x = image_width // 2
                label_pos_y = 30
                print(f"[DEBUG] Drawing label: {label_text} at ({label_pos_x}, {label_pos_y})")
                (text_width, text_height), baseline = cv2.getTextSize(
                    label_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 3)
                text_x = label_pos_x - text_width // 2
                text_y = label_pos_y
                cv2.putText(
                    frame,
                    label_text,
                    (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.0,
                    (0, 255, 0) if recognized_type == "Statyczny" else (255, 0, 0),
                    3,
                    lineType=cv2.LINE_AA
                )
        else:
            static_buffer.clear()
            dynamic_buffer.clear()
            recognized_sign = ''
            recognized_type = ''
            last_prediction = None

        cooldown_frames = max(0, cooldown_frames - 1)

        cv2.imshow('Sign Language Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1748950068.564607   35366 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748950068.567573   35525 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1748950068.642917   35511 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950068.682258   35509 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950068.683969   35510 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950068.685555   35512 inference_feedback_manager.cc:114] Feedback manager requires a model with a sin

[DEBUG] Motion level: 1.221, Smooth motion: 1.221
[DEBUG] Motion level: 1.200, Smooth motion: 1.210
[DEBUG] Motion level: 1.038, Smooth motion: 1.153
[DEBUG] Motion level: 0.975, Smooth motion: 1.109
[DEBUG] Motion level: 1.006, Smooth motion: 1.088
[DEBUG] Motion level: 2.458, Smooth motion: 1.335
[DEBUG] Motion level: 1.323, Smooth motion: 1.360
[DEBUG] Motion level: 1.340, Smooth motion: 1.421
[DEBUG] Motion level: 1.043, Smooth motion: 1.434
[DEBUG] Motion level: 1.181, Smooth motion: 1.469
[DEBUG] Motion level: 0.885, Smooth motion: 1.154
[DEBUG] Motion level: 1.048, Smooth motion: 1.099
[DEBUG] Motion level: 0.974, Smooth motion: 1.026
[DEBUG] Motion level: 1.228, Smooth motion: 1.063
[DEBUG] Motion level: 1.233, Smooth motion: 1.074
[DEBUG] Motion level: 3.189, Smooth motion: 1.534
[DEBUG] Motion level: 1.448, Smooth motion: 1.615
[DEBUG] Motion level: 1.774, Smooth motion: 1.775
[DEBUG] Motion level: 3.503, Smooth motion: 2.230
[DEBUG] Motion level: 2.635, Smooth motion: 2.510


I0000 00:00:1748950072.245314   35432 cuda_dnn.cc:529] Loaded cuDNN version 90300


[STATIC] Predicted: B (conf: 0.17)
[DEBUG] Motion level: 3.022, Smooth motion: 1.843
[DEBUG] Motion level: 3.101, Smooth motion: 2.167
[DEBUG] Motion level: 3.632, Smooth motion: 2.586
[DEBUG] Motion level: 3.409, Smooth motion: 2.968
[DEBUG] Motion level: 3.290, Smooth motion: 3.291
[DEBUG] Motion level: 1.345, Smooth motion: 2.955
[DEBUG] Motion level: 1.401, Smooth motion: 2.615
[DEBUG] Motion level: 1.335, Smooth motion: 2.156
[DEBUG] Motion level: 1.324, Smooth motion: 1.739
[DEBUG] Motion level: 1.378, Smooth motion: 1.356
[DEBUG] Motion level: 1.353, Smooth motion: 1.358
[DEBUG] Motion level: 1.362, Smooth motion: 1.350
[DEBUG] Motion level: 1.316, Smooth motion: 1.346
[DEBUG] Normalized buffer shape: (5, 63)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[STATIC] Predicted: B (conf: 0.50)
[DEBUG] Motion level: 1.410, Smooth motion: 1.364
[DEBUG] Motion level: 1.331, Smooth motion: 1.354
[DEBUG] Motion level: 1.324, Smooth motion: 1.349
[DEBUG] Motion level: 1.370, Smooth motion: 1.350
[

In [5]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Załaduj modele
model_static = load_model('StaticBiLSTM/static_gestures_model.keras')
label_encoder_static = joblib.load('StaticBiLSTM/static_gestures_labels.pkl')

model_dynamic = load_model('DynamicBiLSTM/dynamic_gestures_model.keras')
label_encoder_dynamic = joblib.load('DynamicBiLSTM/dynamic_gestures_labels.pkl')

# Parametry
motion_threshold = 1.8
motion_history = []
motion_history_len = 5
cooldown_frames = 0
cooldown_threshold = 10
recognized_sign = ''
recognized_type = ''
last_prediction = None
static_buffer = []
dynamic_buffer = []

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def normalize_landmarks_buffer(buffer):
    buffer = np.array(buffer)
    if buffer.shape[0] == 0:
        return np.zeros((1, 63))
    wrist_ref = buffer[0, 3:6]  # landmark_1 x,y,z
    norm_buffer = buffer.copy()
    for i in range(1, 21):
        norm_buffer[:, i*3:(i+1)*3] -= wrist_ref
    return norm_buffer

def extract_right_hand_landmarks(results):
    if results.right_hand_landmarks:
        return [coord for lm in results.right_hand_landmarks.landmark for coord in (lm.x, lm.y, lm.z)]
    return None

def get_hand_bbox(landmarks, image_width, image_height):
    xs = [lm.x * image_width for lm in landmarks.landmark]
    ys = [lm.y * image_height for lm in landmarks.landmark]
    x_min, x_max = max(int(min(xs)), 0), min(int(max(xs)), image_width)
    y_min, y_max = max(int(min(ys)), 0), min(int(max(ys)), image_height)
    return (x_min, y_min, x_max, y_max)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_diff = cv2.absdiff(prev_gray, curr_gray)
        motion_level = np.sum(frame_diff) / (frame.shape[0] * frame.shape[1])
        prev_gray = curr_gray.copy()

        motion_history.append(motion_level)
        if len(motion_history) > motion_history_len:
            motion_history.pop(0)
        smooth_motion = np.mean(motion_history)

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)
        image_height, image_width, _ = frame.shape

        landmarks = extract_right_hand_landmarks(results)

        if landmarks:
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
            bbox = get_hand_bbox(results.right_hand_landmarks, image_width, image_height)
            x1, y1, x2, y2 = bbox
            box_color = (0, 255, 0) if recognized_type == "Statyczny" else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)

            if smooth_motion < motion_threshold:
                dynamic_buffer.clear()
                static_buffer.append(landmarks)
                if len(static_buffer) >= 5 and cooldown_frames == 0:
                    norm_static = normalize_landmarks_buffer(static_buffer)
                    if norm_static.shape[0] == 5:
                        norm_static = np.expand_dims(norm_static, axis=0)
                        prediction = model_static.predict(norm_static, verbose=0)[0]
                        label = label_encoder_static.inverse_transform([np.argmax(prediction)])[0]
                        conf = np.max(prediction)
                        if conf > 0.8 and label != last_prediction:
                            recognized_sign = label
                            recognized_type = "Statyczny"
                            last_prediction = label
                            cooldown_frames = cooldown_threshold
                            static_buffer.clear()
                            dynamic_buffer.clear()
            else:
                static_buffer.clear()
                dynamic_buffer.append(landmarks)
                if len(dynamic_buffer) >= 30 and cooldown_frames == 0:
                    norm_dynamic = normalize_landmarks_buffer(dynamic_buffer)
                    if norm_dynamic.shape[0] == 30:
                        norm_dynamic = np.expand_dims(norm_dynamic, axis=0)
                        prediction = model_dynamic.predict(norm_dynamic, verbose=0)[0]
                        label = label_encoder_dynamic.inverse_transform([np.argmax(prediction)])[0]
                        conf = np.max(prediction)
                        if conf > 0.8 and label != last_prediction:
                            recognized_sign = label
                            recognized_type = "Dynamiczny"
                            last_prediction = label
                            cooldown_frames = cooldown_threshold
                            static_buffer.clear()
                            dynamic_buffer.clear()

            if recognized_sign and recognized_type:
                label_text = f'{recognized_sign} ({recognized_type})'
                label_pos_x = image_width // 2
                label_pos_y = 30
                (text_width, _), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 3)
                text_x = label_pos_x - text_width // 2
                cv2.putText(frame, label_text, (text_x, label_pos_y), cv2.FONT_HERSHEY_SIMPLEX,
                            1.0, (0, 255, 0) if recognized_type == "Statyczny" else (255, 0, 0), 3, cv2.LINE_AA)
        else:
            static_buffer.clear()
            dynamic_buffer.clear()
            recognized_sign = ''
            recognized_type = ''
            last_prediction = None

        cooldown_frames = max(0, cooldown_frames - 1)

        cv2.imshow('Sign Language Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1748950510.030400   35366 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1748950510.031475   38505 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics 620 (KBL GT2)
W0000 00:00:1748950510.100607   38497 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950510.136915   38496 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950510.139891   38498 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748950510.139949   38499 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback 

KeyboardInterrupt: 

# Śledzenie odległości punktów

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Załaduj modele
model_static = load_model('StaticBiLSTM/static_gestures_model.keras')
model_dynamic = load_model('DynamicBiLSTM/dynamic_gestures_model.keras')
label_encoder_static = joblib.load('StaticBiLSTM/static_gestures_labels.pkl')
label_encoder_dynamic = joblib.load('DynamicBiLSTM/dynamic_gestures_labels.pkl')

# Parametry

cooldown_frames = 0
cooldown_threshold = 10
recognized_sign = ''
recognized_type = ''
last_prediction = None
static_buffer = []
dynamic_buffer = []

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def normalize_landmarks_buffer(buffer):
    buffer = np.array(buffer)
    if buffer.shape[0] == 0 or buffer.shape[1] != 63:
        return np.zeros((1, 63))  # awaryjnie
    wrist_ref = buffer[0, 3:6]  # landmark_1_x, _y, _z
    norm_buffer = buffer.copy()
    for i in range(1, 21):  # zakres 1 do 20 (czyli i*3 < 63)
        norm_buffer[:, i*3:(i+1)*3] -= wrist_ref
    return norm_buffer

def extract_right_hand_landmarks(results):
    if results.right_hand_landmarks:
        return [coord for lm in results.right_hand_landmarks.landmark for coord in (lm.x, lm.y, lm.z)]
    return None

def get_hand_centroid(landmarks):
    # landmarks: lista 63 floatów (21 punktów x 3)
    points = np.array(landmarks).reshape(21, 3)
    centroid = points[:, :2].mean(axis=0)  # średnia x,y (ignorujemy z)
    return centroid

def get_hand_bbox(landmarks, image_width, image_height):
    xs = [lm.x * image_width for lm in landmarks.landmark]
    ys = [lm.y * image_height for lm in landmarks.landmark]
    x_min, x_max = max(int(min(xs)), 0), min(int(max(xs)), image_width)
    y_min, y_max = max(int(min(ys)), 0), min(int(max(ys)), image_height)
    return (x_min, y_min, x_max, y_max)

def landmarks_movement(prev, curr):
    """
    Oblicza średnią odległość L2 między odpowiadającymi landmarkami
    prev, curr: listy 63 floatów (21 punktów x 3)
    """
    prev_pts = np.array(prev).reshape(21, 3)
    curr_pts = np.array(curr).reshape(21, 3)
    distances = np.linalg.norm(curr_pts - prev_pts, axis=1)  # odległość L2 dla każdego punktu
    return np.mean(distances)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

prev_landmarks = None

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image_rgb)
        image_height, image_width, _ = frame.shape

        landmarks = extract_right_hand_landmarks(results)

        if landmarks:
            mp_drawing.draw_landmarks(frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

            if prev_landmarks is None:
                movement = 0.0
            else:
                movement = landmarks_movement(prev_landmarks, landmarks)
            prev_landmarks = landmarks

            # Próg zmiany pozycji landmarków (dostosuj eksperymentalnie)
            movement_threshold = 0.005

            bbox = get_hand_bbox(results.right_hand_landmarks, image_width, image_height)
            x1, y1, x2, y2 = bbox

            if movement < movement_threshold:
                # Gest statyczny
                dynamic_buffer.clear()
                static_buffer.append(landmarks)
                if len(static_buffer) >= 5 and cooldown_frames == 0:
                    norm_static = normalize_landmarks_buffer(static_buffer)
                    if norm_static.shape[0] == 5:
                        norm_static = np.expand_dims(norm_static, axis=0)
                        prediction = model_static.predict(norm_static, verbose=0)[0]
                        label = label_encoder_static.inverse_transform([np.argmax(prediction)])[0]
                        conf = np.max(prediction)
                        if conf > 0.3:
                            recognized_sign = label
                            recognized_type = "Statyczny"
                            last_prediction = label
                            cooldown_frames = cooldown_threshold
                            static_buffer.clear()
                            dynamic_buffer.clear()
            else:
                # Gest dynamiczny
                static_buffer.clear()
                dynamic_buffer.append(landmarks)
                if len(dynamic_buffer) >= 30 and cooldown_frames == 0:
                    norm_dynamic = normalize_landmarks_buffer(dynamic_buffer)
                    if norm_dynamic.shape[0] == 30:
                        norm_dynamic = np.expand_dims(norm_dynamic, axis=0)
                        prediction = model_dynamic.predict(norm_dynamic, verbose=0)[0]
                        label = label_encoder_dynamic.inverse_transform([np.argmax(prediction)])[0]
                        conf = np.max(prediction)
                        if conf > 0.3:
                            recognized_sign = label
                            recognized_type = "Dynamiczny"
                            last_prediction = label
                            cooldown_frames = cooldown_threshold
                            static_buffer.clear()
                            dynamic_buffer.clear()

            box_color = (0, 255, 0) if recognized_type == "Statyczny" else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)

            if recognized_sign and recognized_type:
                label_text = f'{recognized_sign} ({recognized_type})'
                label_pos_x = image_width // 2
                label_pos_y = 30
                (text_width, _), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 3)
                text_x = label_pos_x - text_width // 2
                cv2.putText(frame, label_text, (text_x, label_pos_y), cv2.FONT_HERSHEY_SIMPLEX,
                            1.0, box_color, 3, cv2.LINE_AA)
        else:
            static_buffer.clear()
            dynamic_buffer.clear()
            recognized_sign = ''
            recognized_type = ''
            last_prediction = None
            prev_landmarks = None

        cooldown_frames = max(0, cooldown_frames - 1)

        cv2.imshow('Sign Language Recognition', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2025-06-03 14:17:12.343758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748953032.362274   46199 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748953032.367852   46199 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748953032.382164   46199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748953032.382186   46199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748953032.382188   46199 computation_placer.cc:177] computation placer alr